In [1]:
####################
#set up packages 
####################
import ee
import pandas as pd
import geemap
import time
from datetime import date
import numpy as np
import os, sys
import seaborn as sns
import matplotlib.pyplot as plt
import geemap.colormaps as cm
import ipywidgets as widgets






##Program to test mapping##

In [3]:

####################################
#country list to loop through
####################################
Map=geemap.Map() #setting up the map to display 
countries = ee.FeatureCollection('FAO/GAUL/2015/level0') #countries is the dataset
 
countrylist=['Bahamas','Barbados','Cuba','Dominican Republic','Haiti','Trinidad and Tobago']#['Jamaica','Puerto Rico',



In [4]:
dropdown = widgets.Dropdown(
    options=countrylist,
    description='Country:'
)
dropdown

Dropdown(description='Country:', options=('Bahamas', 'Barbados', 'Cuba', 'Dominican Republic', 'Haiti', 'Trini…

In [5]:

countryname=dropdown.value
roi = countries.filter(ee.Filter.eq('ADM0_NAME', countryname)) #roi is a subset of the countries dataset 
roigeom=roi.geometry() #set up as a geometry to allow .filterBounds and .clip would work in other ways but this is most efficient
roicenter=roigeom.centroid().getInfo() #this allows the map to be centered on the center of the roi
Map.addLayer(roigeom,{},'Region of Interest')
Map.centerObject(roigeom)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…